In [3]:
import csv
import pandas as pd
import spacy
import re
import string
import pyLDAvis
import pyLDAvis.sklearn
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from sklearn.decomposition import LatentDirichletAllocation, NMF

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

nlp = spacy.load("en_core_web_sm")

In [4]:
df = pd.read_csv('genshintwittermaintenanceone.csv')
df = df.drop(columns = ['Unnamed: 0', 'outlinks', 'tcooutlinks'])

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

def cleanurlhashtags(input):

    text = re.sub(r"http\S+", "", input)
    text = re.sub(r"#\S+", "", text)
    return text

def tostring(input):
    return str(input)

df['content'] = df['content'].apply(tostring)
df['content'] = df['content'].apply(cleanurlhashtags)
df['content'] = df['content'].map(alphanumeric).map(punc_lower)
df['content'] = df['content'].fillna(value='')

content = df['content'].to_list()

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = ENGLISH_STOP_WORDS,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(content)

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(content)

lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)

lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=20, random_state=0)

In [5]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

2020-11-14 20:17:44,374 : INFO : NumExpr defaulting to 8 threads.


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16     0.175162 -0.101586       1        1  9.354839
3      0.314701  0.180825       2        1  8.392879
19     0.024920 -0.194393       3        1  6.201130
4     -0.159836  0.096498       4        1  5.488048
8     -0.066532 -0.084243       5        1  5.228480
18     0.129726 -0.058834       6        1  5.142079
1      0.073171 -0.142476       7        1  4.845369
2     -0.010972 -0.175046       8        1  4.791230
12    -0.141121  0.048778       9        1  4.755737
13    -0.079944  0.000832      10        1  4.538980
5     -0.021594  0.009830      11        1  4.494127
10    -0.120522  0.041663      12        1  4.376138
6     -0.198976  0.166286      13        1  4.321686
17    -0.141606  0.061419      14        1  4.317027
15     0.105333  0.206260      15        1  4.213486
14    -0.017199 -0.004194      16        1  4.208007
9      0.022656 -0.161214      17        1  3.970880
11    -0.077137 -0.030355      18        1  3.870751
7      0.194781  0.129487      19        1  3.774771
0     -0.005013  0.010462      20        1  3.714356, topic_info=               Term         Freq        Total Category  logprob  loglift
442         genshin  1821.000000  1821.000000  Default  30.0000  30.0000
554          impact  1246.000000  1246.000000  Default  29.0000  29.0000
174          childe   629.000000   629.000000  Default  28.0000  28.0000
444   genshinimpact   512.000000   512.000000  Default  27.0000  27.0000
577            just   708.000000   708.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
1076            tho    25.485770    48.300836  Topic20  -4.4290   2.6536
1227      yesterday    22.474576    44.255072  Topic20  -4.5547   2.6154
1134           used    20.816567    44.833907  Topic20  -4.6314   2.5257
704       mondstadt    19.499001    84.071316  Topic20  -4.6968   1.8317
721            need    20.617067   228.660476  Topic20  -4.6410   0.8868

[974 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         5  0.154053     able
0         8  0.044015     able
0        11  0.242083     able
0        18  0.550189     able
3         3  0.090265    abyss
...     ...       ...      ...
1229     12  0.055387    zhong
1229     14  0.526175    zhong
1229     17  0.055387    zhong
1230      9  0.908620  zhongli
1230     13  0.087764  zhongli

[2315 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 4, 20, 5, 9, 19, 2, 3, 13, 14, 6, 11, 7, 18, 16, 15, 10, 12, 8, 1])

In [6]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3      0.317371  0.000408       1        1  7.627657
16     0.243748  0.064888       2        1  7.144009
13    -0.038299 -0.104575       3        1  5.682947
4     -0.090035 -0.099098       4        1  5.505659
12    -0.091275 -0.025315       5        1  5.426240
18     0.107678 -0.085555       6        1  5.123380
10    -0.040468 -0.015016       7        1  5.115703
8     -0.077134 -0.015179       8        1  5.089577
2     -0.002490  0.112320       9        1  4.801930
1      0.031255  0.123206      10        1  4.738196
5     -0.030620 -0.054115      11        1  4.713419
0     -0.064225 -0.096598      12        1  4.680656
9     -0.004954 -0.003418      13        1  4.641080
15     0.003409 -0.142328      14        1  4.600473
19     0.003033 -0.039288      15        1  4.397155
14    -0.027215 -0.097168      16        1  4.296748
6     -0.162501  0.069407      17        1  4.279886
7      0.084170  0.048131      18        1  4.108600
11    -0.059121  0.156310      19        1  4.056427
17    -0.102327  0.202980      20        1  3.970260, topic_info=         Term        Freq       Total Category  logprob  loglift
629      live  174.000000  174.000000  Default  30.0000  30.0000
585      klee  164.000000  164.000000  Default  29.0000  29.0000
282     diluc  139.000000  139.000000  Default  28.0000  28.0000
760    paimon   85.000000   85.000000  Default  27.0000  27.0000
202      come  145.000000  145.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
237    credit    9.833585   22.309463  Topic20  -4.5233   2.4071
1181     week   10.788706   39.163180  Topic20  -4.4306   1.9371
645       lot    9.175755   34.174148  Topic20  -4.5926   1.9114
640   looking    8.435218   35.745336  Topic20  -4.6767   1.7823
448   getting    8.480994   47.301280  Topic20  -4.6713   1.5076

[987 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2        16  0.836096      abt
3         4  0.309284    abyss
3        18  0.618569    abyss
5         1  0.026534  account
5         2  0.132669  account
...     ...       ...      ...
1230     16  0.039847  zhongli
1230     17  0.111570  zhongli
1230     19  0.047816  zhongli
1230     20  0.007969  zhongli
1231      5  0.895327   zombie

[2645 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 17, 14, 5, 13, 19, 11, 9, 3, 2, 6, 1, 10, 16, 20, 15, 7, 8, 12, 18])

In [7]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16     0.169448  0.124336       1        1  9.354839
3      0.190145 -0.445421       2        1  8.392879
19    -0.039073 -0.091061       3        1  6.201130
4     -0.107253  0.434978       4        1  5.488048
8      0.331226 -0.233488       5        1  5.228480
18     0.317201 -0.074551       6        1  5.142079
1      0.110261  0.379690       7        1  4.845369
2     -0.284002 -0.307625       8        1  4.791230
12    -0.378889  0.206916       9        1  4.755737
13     0.264210  0.346508      10        1  4.538980
5      0.358716  0.194240      11        1  4.494127
10    -0.078545  0.246390      12        1  4.376138
6     -0.474218  0.018804      13        1  4.321686
17    -0.266764  0.322002      14        1  4.317027
15    -0.011917 -0.429156      15        1  4.213486
14    -0.225480  0.029697      16        1  4.208007
9     -0.127201 -0.292436      17        1  3.970880
11    -0.367464 -0.162431      18        1  3.870751
7      0.157403 -0.284149      19        1  3.774771
0      0.462195  0.016757      20        1  3.714356, topic_info=               Term         Freq        Total Category  logprob  loglift
442         genshin  1821.000000  1821.000000  Default  30.0000  30.0000
554          impact  1246.000000  1246.000000  Default  29.0000  29.0000
174          childe   629.000000   629.000000  Default  28.0000  28.0000
444   genshinimpact   512.000000   512.000000  Default  27.0000  27.0000
577            just   708.000000   708.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
1076            tho    25.485770    48.300836  Topic20  -4.4290   2.6536
1227      yesterday    22.474576    44.255072  Topic20  -4.5547   2.6154
1134           used    20.816567    44.833907  Topic20  -4.6314   2.5257
704       mondstadt    19.499001    84.071316  Topic20  -4.6968   1.8317
721            need    20.617067   228.660476  Topic20  -4.6410   0.8868

[974 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         5  0.154053     able
0         8  0.044015     able
0        11  0.242083     able
0        18  0.550189     able
3         3  0.090265    abyss
...     ...       ...      ...
1229     12  0.055387    zhong
1229     14  0.526175    zhong
1229     17  0.055387    zhong
1230      9  0.908620  zhongli
1230     13  0.087764  zhongli

[2315 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 4, 20, 5, 9, 19, 2, 3, 13, 14, 6, 11, 7, 18, 16, 15, 10, 12, 8, 1])

In [8]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
16     -10.878068   10.538728       1        1  9.354839
3      -60.884026  -75.591820       2        1  8.392879
19      -2.336119  -53.289074       3        1  6.201130
4      -61.385319  -12.914634       4        1  5.488048
8      -28.737743  126.296997       5        1  5.228480
18       1.066606 -123.552811       6        1  5.142079
1       45.140720   -7.947869       7        1  4.845369
2     -140.795975  -98.530548       8        1  4.791230
12      61.531681  -83.240562       9        1  4.755737
13     -72.563057 -140.396881      10        1  4.538980
5       15.984491   63.578186      11        1  4.494127
10    -160.851624   68.945145      12        1  4.376138
6       85.961830   51.922935      13        1  4.321686
17     -99.020836  109.936348      14        1  4.317027
15    -176.764130   -8.681684      15        1  4.213486
14    -117.189880  -37.420200      16        1  4.208007
9      108.829918  -22.946203      17        1  3.970880
11      47.588097  121.934814      18        1  3.870751
7     -106.467834   30.176889      19        1  3.774771
0      -50.613804   59.328842      20        1  3.714356, topic_info=               Term         Freq        Total Category  logprob  loglift
442         genshin  1821.000000  1821.000000  Default  30.0000  30.0000
554          impact  1246.000000  1246.000000  Default  29.0000  29.0000
174          childe   629.000000   629.000000  Default  28.0000  28.0000
444   genshinimpact   512.000000   512.000000  Default  27.0000  27.0000
577            just   708.000000   708.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
1076            tho    25.485770    48.300836  Topic20  -4.4290   2.6536
1227      yesterday    22.474576    44.255072  Topic20  -4.5547   2.6154
1134           used    20.816567    44.833907  Topic20  -4.6314   2.5257
704       mondstadt    19.499001    84.071316  Topic20  -4.6968   1.8317
721            need    20.617067   228.660476  Topic20  -4.6410   0.8868

[974 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         5  0.154053     able
0         8  0.044015     able
0        11  0.242083     able
0        18  0.550189     able
3         3  0.090265    abyss
...     ...       ...      ...
1229     12  0.055387    zhong
1229     14  0.526175    zhong
1229     17  0.055387    zhong
1230      9  0.908620  zhongli
1230     13  0.087764  zhongli

[2315 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 4, 20, 5, 9, 19, 2, 3, 13, 14, 6, 11, 7, 18, 16, 15, 10, 12, 8, 1])